# BioscrapeCOBRA: a hyrid composite of chemical reaction networks with flux balance analysis

## Overview
This notebook combines a chemical reaction network model with a flux balance analysis model, and a physics engine

* SBML
* BiGG model

In [ ]:
#@title install vivarium libraries

#Uncomment in Colab Below

"""
%%capture
!pip install git+https://github.com/vivarium-collective/vivarium-notebooks
!pip install vivarium-core==0.1.5
!pip install vivarium-cobra==0.0.6
!pip install vivarium-pymunk==0.0.2
!pip install vivarium-bioscrape
"""

In [ ]:
#@title imports
import numpy as np
import pylab as plt

# process/composition interface
from vivarium.core.process import (
    Process, Deriver, Composite,
)

# helper functions for composition
from vivarium.core.composition import (
    simulate_process_in_experiment,
    simulate_compartment_in_experiment,
    compartment_in_experiment,
    compose_experiment,
    FACTORY_KEY,
)

# other vivarium imports
from vivarium.core.experiment import Experiment, pp
from vivarium.library.units import units

# vivarium plotting functions
from vivarium.plots.simulation_output import plot_simulation_output
from vivarium.plots.agents_multigen import plot_agents_multigen
from vivarium.plots.topology import plot_topology

# import processes and composites
from vivarium.processes.tree_mass import TreeMass
from vivarium.processes.divide_condition import DivideCondition
from vivarium.processes.meta_division import MetaDivision

# vivarium-cobra processes
from vivarium_cobra.processes.volume import Volume
from vivarium_cobra.composites.cobra_composite import CobraComposite
from vivarium_cobra.processes.dynamic_fba import (
    DynamicFBA, 
    get_iAF1260b_config, 
    print_growth
)

# vivarium-bioscrape processes
from vivarium_bioscrape.processes.bioscrape import Bioscrape


In [ ]:
# plotting function for metabolism output
def plot_metabolism(data, ncol=2):
    original_fontsize = plt.rcParams['font.size']
    plt.rcParams.update({'font.size': 9})
    
    # initialize subplots
    n_rows = 2
    n_cols = 2
    fig = plt.figure(figsize=(n_cols * 7, n_rows * 3))
    grid = plt.GridSpec(n_rows, n_cols)
    
    time_vec = data['time']
    
    # mass
    ax = fig.add_subplot(grid[0, 0])  
    ax.plot(time_vec, data['global'][('mass', 'femtogram')], label = 'mass')
    ax.set_title('total compartment mass (fg)')
    ax.set_xlabel('time (sec)')
#   ax.legend(loc='center left', bbox_to_anchor=(1.0, 0.5), ncol=ncol)
    
    # external
    ax = fig.add_subplot(grid[0, 1])  
    for mol_id, series in data['external'].items():
        if sum(series) != 0.0:
            ax.plot(time_vec, series, label = mol_id)
    ax.set_title('external concentrations (log)')
    ax.set_yscale('log')
    ax.legend(loc='center left', bbox_to_anchor=(1.0, 0.5), ncol=ncol)
    
    # internal
    ax = fig.add_subplot(grid[1, 1])  
    for mol_id, series in data['internal_counts'].items():
        if sum(series) != 0.0:
            ax.plot(time_vec, series, label = mol_id)
    ax.set_title('internal molecule counts (log)')
    ax.set_xlabel('time (sec)')
    ax.set_yscale('log')
    fig.tight_layout()
    plt.rcParams.update({'font.size': original_fontsize})

# Flux Balance Analysis with COBRA

 * This process is a wrapper for the popular FBA solver [COBRA](https://cobrapy.readthedocs.io/en/latest/).
 * Metabolism is initialized with a [BiGG metabolic model](http://bigg.ucsd.edu) called *iAF1260b*, which has 2382 reactions, 1261 genes, and 166 metabolites.

In [ ]:
compartment_in_experiment
# get the configuration for the iAF1260b BiGG model
cobra_config = get_iAF1260b_config()
cobra_config.update({'time_step': 10})
config = {'cobra': cobra_config}   

# load it into DynamicFBA
metabolism = CobraComposite(config)

# get the model's initial state
initial_state = metabolism.initial_state({}) 

# run simulation
sim_settings = {
    'initial_state': initial_state,
    'total_time': 500}
cobra_timeseries = simulate_compartment_in_experiment(metabolism, sim_settings)

In [ ]:
print_growth(cobra_timeseries['global'])
plot_metabolism(cobra_timeseries)

# Chemical Reaction Network with SBML 

In [ ]:


# initialize Bioscrape process
time_step = 10

#This will configure a deterministic model as a system of ODEs
deterministic_bioscrape_config = {
            'sbml_file': 'LacOperon_deterministic.xml',
            'stochastic': False,
            'initial_volume': 1,
            'internal_dt': 0.01,
            'time_step':time_step}


bioscrape_process_deterministic = Bioscrape(deterministic_bioscrape_config)

In [ ]:
#Run the simulation
total_time = 10000

# initial state
initial_state = bioscrape_process_deterministic.initial_state()

# run simulation
settings = {
    'total_time': total_time,
    'timestep':time_step,
    'initial_state': initial_state,
    'display_info': False,
    'progress_bar': False}
bioscrape_timeseries_deterministic = simulate_process_in_experiment(bioscrape_process_deterministic, settings)



In [ ]:
fig = plot_simulation_output(bioscrape_timeseries_deterministic)

In [ ]:
#This will configure a stochastic model using the Gillespie SSA algorithm
stochastic_bioscrape_config = {
            'sbml_file': 'LacOperon_stochastic.xml',
            'stochastic': True,
            'safe_mode':True,
            'initial_volume': 1,
            'internal_dt': 0.1,
            'time_step':time_step}


bioscrape_process_stochastic = Bioscrape(stochastic_bioscrape_config)

In [ ]:
#Run the simulation
total_time = 10000

# initial state
initial_state = bioscrape_process_stochastic.initial_state()
print(initial_state)

# run simulation
settings = {
    'total_time': total_time,
    'initial_state': initial_state,
    'display_info': False,
    'progress_bar': False}
bioscrape_timeseries_stochastic = simulate_process_in_experiment(bioscrape_process_stochastic, settings)

print("Complete!")

In [ ]:
fig = plot_simulation_output(bioscrape_timeseries_stochastic)

# Multibody physics with pymunk

In [ ]:
from vivarium_multibody.processes.multibody_physics import test_growth_division
from vivarium_multibody.processes.multibody_physics import agent_body_config, volume_from_length
from vivarium_multibody.plots.snapshots import (
    plot_snapshots,
    format_snapshot_data,
    make_snapshots_figure,
    get_field_range,
    get_agent_colors,
)
import numpy as np

from ipywidgets import interactive
import ipywidgets as widgets

In [ ]:
total_time = 120
n_agents = 2
agent_ids = [
    str(agent_id)
    for agent_id in range(n_agents)]

# configure the multibody process
multibody_bounds = [40, 40]
multibody_config = {
    'bounds': multibody_bounds}
body_config = {
    'bounds': multibody_bounds,
    'agent_ids': agent_ids}
multibody_config.update(agent_body_config(body_config))

# experiment settings
experiment_settings = {'progress_bar': False, 'display_info': False}

# run multibody with grow/divide agents
multibody_data = test_growth_division(
        config=multibody_config,
        growth_rate=0.05,
        growth_rate_noise=0.05,
        division_volume=volume_from_length(4, 1),
        total_time=total_time,
        experiment_settings=experiment_settings)

multibody_agents, multibody_fields = format_snapshot_data(multibody_data)

In [ ]:
try:
    time_index = 1
    time_indices = np.array([time_index])
    time_vec = list(agents.keys()) 

    # time_indices = np.round(np.linspace(0, len(time_vec) - 1, n_snapshots)).astype(int)
    snapshot_times = [time_vec[i] for i in time_indices]

    # get fields id and range
    multibody_field_range = get_field_range(fields, time_vec)

    # get agent ids
    multibody_agent_colors = get_agent_colors(multibody_agents)


    time_index_range = len(time_vec) -1

    def plot_single_snapshot(t_index):   
        time_indices = np.array([t_index])
        snapshot_time = [time_vec[t_index]]
        fig = make_snapshots_figure(
            time_indices=time_indices,
            snapshot_times=snapshot_time,
            agents=multibody_agents,
            agent_colors=multibody_agent_colors,
            fields=multibody_fields,
            field_range=multibody_field_range,
            n_snapshots=1,
            bounds=multibody_bounds,
            default_font_size=12,
            plot_width=6,
        )
except NameError as e:
    print("Something isn't working! Error:", e)

In [ ]:
try:
    interactive_plot = interactive(
        plot_single_snapshot, 
        t_index=widgets.IntSlider(min=0, max=time_index_range, step=1, value=0))
    interactive_plot
except NameError as e:
    print("Something isn't working! Error:", e)

# BioscrapeCOBRA: a CRN/FBA Composite

This also includes several adaptor "Deriver" Processes, such as FluxDeriver and BiomassAdaptor

* BiomassAdaptor demonstrates vivarium units, and also the use of a Deriver as an adaptor between Processes

## Flux Deriver

A Deriver is a kind of Process which can be used to convert values of one store into another type of value in another store. For example, this FluxDeriver converts amounts of species and changes in amounts of species to instantaneous fluxes.

This FluxDeriver is used in the composite to convert the changes of Lactose and Glucose from the CRN model into flux-bounds for the FBA model. In these cases "deltas" (changes in Species) are converted to fluxes by dividing by the timestep.

This FluxDeriver is also used to convert changes in biomass, output from the FBA model, into a dilution rate constant used by the CRN. This flux is computered differently because the change in biomass each timestep is not stored. Additionally, dilution rate must be calculated as a percentage of the current amount, not absolute flux, and be non-negative.

In [ ]:
class FluxDeriver(Process):
    """ Bioscrape delta species to fluxes for constraining COBRA """
    
    defaults = {
        'time_step':1.0,
        'flux_keys': {}, #key --> {option dictionary}
        'default_options' : { #default options if option_dictionary is empty
            "input_type":"delta", #"delta" corresponds to changes between timesteps. "amount" corresponds to the absolute quantity
        }
    }
    def __init__(self, config = None):
        super().__init__(config)
        
        self.prev_inputs = {k:None for k in self.parameters["flux_keys"]}

    def initial_state(self, config=None):
        return {}

    def ports_schema(self):
        return {
            'inputs': {
                flux_key: {
                    '_default': 0.0}
                for flux_key in self.parameters['flux_keys']
            },
            'fluxes': {
                flux_key: {
                    '_updater': 'set',
                    '_emit':True }
                for flux_key in self.parameters['flux_keys']
            }
        }
    
    def compute_flux(self, flux_key, dt, inputs):
        #computes the flux for a specific flux key of the time interval dt
        
        input_type = self.parameters['flux_keys'][flux_key].get("input_type", self.parameters['default_options']["input_type"])
        
        #Set delta
        if input_type == "delta":
            delta = inputs[flux_key]
        elif input_type == 'amount':
            if self.prev_inputs[flux_key] is None:
                delta = 0
            else:
                delta = inputs[flux_key]-self.prev_inputs[flux_key]
        else:
            raise ValueError(f"Unknown input_type: {input_type} for flux_key {flux_key}")
        
        return delta / dt
     
    def next_update(self, timestep, states):
        inputs = states["inputs"]
        
        update = {}
        update['fluxes'] = {}
        
        for flux_key in self.parameters['flux_keys']:
            #flux_dt = self.get_flux_interval(flux_key, timestep)
            #if flux_dt is not None:
            update['fluxes'][flux_key] = self.compute_flux(flux_key, timestep, inputs)
            
        self.prev_inputs = inputs
        return update

    
class DilutionFluxDeriver(FluxDeriver):
    #This is a non-negative "percentage" flux used to control a dilution rate of a CRN from a the rate of growth in biomass
    
    def next_update(self, timestep, states):        
        inputs = states["inputs"]
        
        update = {}
        update['fluxes'] = {}
        
        for flux_key in self.parameters['flux_keys']:
            flux = self.compute_flux(flux_key, timestep, inputs)

            #Convert to a percent of the previous input
            if self.prev_inputs[flux_key] is None or (self.prev_inputs[flux_key] == 0 and states["inputs"][flux_key] == 0):
                #Edge case: If both the current amount and the previous amount were 0, return 0
                update['fluxes'][flux_key] = 0

            elif self.prev_inputs[flux_key] == 0 and states["inputs"][flux_key] > 0:
                #Edge case: If both the previous amount was 0, us the current amount for the normalization
                update['fluxes'][flux_key] = flux/states["inputs"][flux_key]

            else:
                #Standard case
                update['fluxes'][flux_key] = flux/self.prev_inputs[flux_key]

            #Enforce non-negativity
            if update['fluxes'][flux_key] < 0 :
                update['fluxes'][flux_key] = 0

                
        self.prev_inputs = inputs
        return update

class AverageFluxDeriver(FluxDeriver):
    #This is similar to a FluxDeriver, but fluxes are averaged between updates. This is useful to compute fluxes 
    #which are used by a process with a larger dt than the processes controlling the flux, for example a stochastic
    #CRN may be updated frequently but FBA may be updated less frequently.
    pass

## BiomassAdaptor Deriver

This adaptor is used to do unit conversion between the mass output from cobra and a Species used in the Bioscrape model to represent biomass.

In [ ]:
class mass_to_concentration(Deriver):
    """ Adapts COBRA mass variable to Bioscrape biomass """
    
    defaults = {
        'input_mass_units':1.0 * units.fg,
        'input_volume_units':1.0 * units.fL,
        'output_concentration_units':1.0 *units.mmolar,
        'characteristic_output_volume':1.0 *units.fL,
        'mass_species_molecular_weight':1.0 *units.fg/units.molec
    }
    
    def initial_state(self, config=None):
        return {}

    def ports_schema(self):
        return {
            'input': {
                'mass': {
                    '_default': 1.0 * self.parameters["input_mass_units"],
                },
                'volume': {
                    '_default': 1.0 * self.parameters["input_volume_units"]}
            },
            'output': {
                # the value used by Bioscrape
                'biomass': {
                    '_default': 1.0,
                    '_update': 'set',
                }
            }
        }


    def next_update(self, timestep, states):
        mass = states['input']['mass']
        
        # do conversion
        #Concentration = mass/molecular_weight/characteristic volume
        #Note: Biomass is also used to set Volume, so here we just set the scale
        mass_species_conc = mass/self.config['mass_species_molecular_weight']/(self.config['characteristic_output_volume'])        
    
        update = {
            'output': {
                #Return the correct units, with units stripped away for bioscrape
                'biomass': mass_species_conc.to(self.config['output_concentration_units']).magnitude
                }
            }
        return update

class mass_to_count(Deriver):
    """ Adapts COBRA mass variable to Bioscrape biomass """
    
    defaults = {
        'input_mass_units':1.0 * units.fg,
        'mass_species_molecular_weight':1.0 *units.fg/units.molec
    }
    
    def initial_state(self, config=None):
        return {}

    def ports_schema(self):
        return {
            'input': {
                'mass': {
                    '_default': 1.0 * units.fg,
                },
            },
            'output': {
                # the value used by Bioscrape
                'biomass': {
                    '_default': 1.0,
                    '_update': 'set',
                }
            }
        }


    def next_update(self, timestep, states):
        mass = states['input']['mass']
        
        # do conversion
        #count = mass/molecular_weight
        #Note: Biomass is also used to set Volume, so here we just set the scale
        mass_species_count = mass/self.config['mass_species_molecular_weight']        
    
        update = {
            'output': {
                #Return the correct units, with units stripped away for bioscrape
                'biomass': mass_species_count.magnitude
                }
            }
        return update

# BioscrapeCOBRA Composite configurations

This composite demonstrates advanced generate_topology() methods, which split ports across multiple stores with `'_path'`

TODO:
    * units for Biomass is in mmol/L of a species with mw of 1 fg/count = avagadro's number * fg / mol
    * import of Glucose is also in terms of 'Biomass' species
    * stochastically, it is in counts. Volume
    * logic for counts/concentrations if stochastic/deterministic
    * Bioscrape species 'Lactose_internal', 'Lactose_external' need to get wired to ('external',),
        But don't let Bioscrape update them. Only use FBA's resulting uptake.


In [ ]:
#choose the SBML file and set other bioscrape parameters
deterministic_bioscrape_config = {
            'sbml_file': 'LacOperon_deterministic.xml',
            'stochastic': False,
            'initial_volume': 1,
            'internal_dt': 0.01,}

# set cobra constrained reactions config
cobra_config = get_iAF1260b_config()
cobra_config.update({'time_step': 10})

#set up the config for the FluxDeriver
flux_config = {
    'flux_keys': {
        'Lactose_consumed': {}, #No options specified
        'Glucose_internal': {}  #No options specified
    },
}

dilution_rate_flux_config = {
    'time_step':cobra_config['time_step'],
    'flux_keys': {
        'biomass':{
            "input_type":"amount"
        }
    }
}

#Here we override the default ports schema of the Biomass species and the k_dilution rate in Bioscrape.
#This is done so they can be set by the Derivers connected to mass and mass flux from Cobra.
schema_override = {
    'bioscrape': {
        'species': {
            'Biomass': { 
                '_updater': 'set'  #override bioscrape ('species', 'Biomass') with a 'set' updater
            }
        },
        'rates':{
            'k_dilution__':{
                '_emit':True, #k_dilution should be emitted so it can be plotted
                '_updater':'set'
            }
        }
    }
}

In [ ]:
class BioscrapeCOBRA(Composite):

    defaults = {
        'bioscrape': deterministic_bioscrape_config,
        'cobra': cobra_config,
        'flux_deriver': flux_config,
        'dilution_rate_flux':dilution_rate_flux_config,
        'divide_on': False, # is division turned on?
        'agent_id': np.random.randint(0, 100),
        'divide_condition': {
            'threshold': 2000 * units.fg},
        'agents_path': ('..', '..', 'agents',),
        'daughter_path': tuple(),
         '_schema': schema_override,
        'stochastic':False
    }

    def generate_processes(self, config):
        processes = {
            'bioscrape': Bioscrape(config['bioscrape']),
            'cobra': DynamicFBA(config['cobra']),
            'mass_deriver': TreeMass(),
            'volume_deriver': Volume(),
        }
        
        #Process Logic for different kinds of simulations
        
        #Deterministic case
        if not config['bioscrape']['stochastic']:
            #deterministic simulations have a variable dilution rate
            processes['dilution_rate_adaptor'] = DilutionFluxDeriver(config["dilution_rate_flux"])
            
            #flux is computed as an instaneous flux
            processes['flux_deriver'] = FluxDeriver(config['flux_deriver'])
            
            #biomass is converted to a concentration
            processes['biomass_adaptor'] = mass_to_concentration()
        
        #Stochastic Case
        else:
            #flux is computed as an average flux
            processes['flux_deriver'] = AverageFluxDeriver(config['flux_deriver'])
            
            #biomass is converted to a molecular count
            processes['biomass_adaptor'] = mass_to_count()
        
        #Division Logic
        if config['divide_on']:
            # division config
            daughter_path = config['daughter_path']
            agent_id = config['agent_id']
            division_config = dict(
                config.get('division', {}),
                daughter_path=daughter_path,
                agent_id=agent_id,
                generator=self)

            processes.update({
                'divide_condition': DivideCondition(config['divide_condition']),
                'division': MetaDivision(division_config)
            })
        return processes

    def generate_topology(self, config):

        topology = {
            'bioscrape': {
                # all species go to a species store on the base level,
                # except Biomass, which goes to the 'globals' store, with variable 'biomass'
                'species': {
                    '_path': ('species',),
                    'Biomass': ('..', 'globals', 'biomass'),
                },
                'delta_species': ('delta_species',),
                'rates': {
                    '_path' : ('rates',),
                },
                'globals': ('globals',),
            },
            'cobra': {
                'internal_counts': ('internal_counts',),
                'external': ('external',),
                'exchanges': ('exchanges',),
                'reactions': ('reactions',),
                'flux_bounds': ('flux_bounds',),
                'global': ('globals',),
            },
            'flux_deriver': {
                'inputs': ('delta_species',),
                #'amounts': ('globals',),
                # connect Bioscrape deltas 'Lactose_consumed' and 'Glucose_internal'
                # to COBRA flux bounds 'EX_lac__D_e' and 'EX_glc__D_e'
                
                'fluxes':{
                        '_path': ('flux_bounds',),
                        'Lactose_consumed': ('EX_lac__D_e',),
                        'Glucose_internal': ('EX_glc__D_e',),
                    }
            },
            
            'mass_deriver': {
                'global': ('globals',),
            },
            'volume_deriver': {
                'global': ('globals',),
            },
            'biomass_adaptor': {
                'input': ('globals',),
                'output': ('globals',),
            }
        }
        
        #Ports added only in the deterministic case
        if not config['stochastic']:
            #Create port biomass flux to the dilution rate computed by the dilution_rate_adaptor process
            topology['dilution_rate_adaptor']= {
                'inputs':('globals',),
                'fluxes':{
                    '_path':('rates',),
                    'biomass':('k_dilution__',)
                }
            }
            
        #Ports added only in the stochastic case
        else:
            pass
        
        if config['divide_on']:
            agents_path = config['agents_path']

            # connect divide_condition to the mass variable
            topology.update({
                'divide_condition': {
                    'variable': ('globals', 'mass',),
                    'divide': ('globals', 'divide',),
                },
                'division': {
                    'global': ('globals',),
                    'agents': agents_path,
                },
            })
        return topology

In [ ]:
bioscrape_cobra_composite = BioscrapeCOBRA({})

#experiment = compartment_in_experiment(bioscrape_cobra_composite)
#pp(experiment.state.get_config(True))



In [ ]:
# get initial state
initial_state = bioscrape_cobra_composite.initial_state()
## override FBA default external state
initial_state['external']['glc__D_e'] = 10
initial_state['external']['lcts_e'] = 10
#TODO Bioscrape and Cobra's external state's should be linked up.

# simulate
settings = {
    'initial_state': initial_state,
    'total_time': 500}
bioscrape_cobra_output = simulate_compartment_in_experiment(
    bioscrape_cobra_composite, 
    settings)

#plt.figure()
#plt.plot(bioscrape_cobra_output['time'], bioscrape_cobra_output['rates']['k_dilution__'])

In [ ]:
fig = plot_simulation_output(bioscrape_cobra_output)

# Stochastic Bioscrape-Cobra

Changes required for Stochastic Model
* New CRN Model without dilution and with initial conditions in counts instead of concentration
    * this means "k_dilution__" porting needs to removed from the topology
* BiomassDeriver converts to counts instead of concentration
* FluxDeriver run in "stochastic" mode where flux is averaged over the cobra update time-frame.

In [ ]:
stochastic_bioscrape_config = {
            'sbml_file': 'LacOperon_stochastic.xml',
            'stochastic': True,
            'initial_volume': 1,
            'internal_dt': 0.01,}

bioscrape_cobra_composite_stochastic = BioscrapeCOBRA({"bioscrape":stochastic_bioscrape_config})
# get initial state
initial_state = bioscrape_cobra_composite_stochastic.initial_state()
## override FBA default external state
initial_state['external']['glc__D_e'] = 10**6
initial_state['external']['lcts_e'] = 10**6

# simulate
settings = {
    'initial_state': initial_state,
    'total_time': 1000}
bioscrape_cobra_output_stochastic = simulate_compartment_in_experiment(
    bioscrape_cobra_composite_stochastic, 
    settings)

In [ ]:
fig = plot_simulation_output(bioscrape_cobra_output_stochastic)

# Turning on Division

In [ ]:
total_time = 2500

agent_id = '1'
division_config = {
    'divide_on': True,
    'agent_id': agent_id,
    'agents_path': ('..', '..', 'agents',),
}
division_composite = BioscrapeCOBRA(division_config)

# initial state
initial_state = division_composite.initial_state()
# initial_state['species']['Glucose_external'] = 2000
initial_state['external']['glc__D_e'] = 10
initial_state['external']['lcts_e'] = 10
initial_state = {
    'agents': {
        agent_id: initial_state}}

# run simulation
# simulate
settings = {
    'outer_path': ('agents', agent_id,),
    'initial_state': initial_state,
    'experiment_id': 'division'}
division_experiment = compartment_in_experiment(
    division_composite,
    settings=settings,
    initial_state=initial_state)



In [ ]:
# run the experiment and extract the data
division_experiment.update(total_time)
division_output = division_experiment.emitter.get_data_unitless()

In [ ]:
plot_settings = {
#     'skip_paths': [('external',)]
    'include_paths': [
        ('globals', 'mass'),
        ('species', 'Glucose_external'),
        ('species', 'Lactose_external'),
        ('flux_bounds', 'EX_glc__D_e'),
        ('flux_bounds', 'EX_lac__D_e'),
        ('internal_counts', 'zn2_c'),
        ],
}
fig = plot_agents_multigen(
    division_output,
    settings=plot_settings,
)

# Simulate BioscrapeCOBRA in a Spatial Environment with pymunk
 * We here plug BioscrapeCOBRA into an environmental composite called Lattice, which has a multi-body physics process (MBP) and a spatial diffusion process (DFF).

In [ ]:
from vivarium_multibody.composites.lattice import Lattice, make_lattice_config

# declare configurations
agent_id = '1'
lattice_bounds = [20, 20]
env_config = {
    'time_step': 60,
    'bounds': lattice_bounds}

# declare the hierarchy
hierarchy = {
    FACTORY_KEY: {
        'type': Lattice,
        'config': make_lattice_config()},
    'agents': {
        agent_id: {
            FACTORY_KEY: {
                'type': BioscrapeCOBRA,
                'config': {
                    'divide_on': True,
                    'agent_id': agent_id,
                    'agents_path': ('..', '..', 'agents',),
                }}}}}


# configure experiment with helper function compose_experiment()
initial_state = division_composite.initial_state()
# initial_state['species']['Glucose_external'] = 2000
initial_state['external']['glc__D_e'] = 10
initial_state['external']['lcts_e'] = 10
initial_state = {
    'agents': {
        agent_id: initial_state}}

experiment_settings = {
    'initial_state': initial_state,
    'experiment_id': 'spatial_environment'}

In [ ]:
# make the experiment
spatial_experiment = compose_experiment(
    hierarchy=hierarchy, 
    settings=experiment_settings)

# run the experiment
spatial_experiment.update(1000)

# retrieve the data
spatial_data = spatial_experiment.emitter.get_data_unitless()
agents, fields = format_snapshot_data(spatial_data)

In [ ]:
fig = plot_snapshots(
    bounds=lattice_bounds, 
    agents=agents, 
    fields=fields)